In [4]:
import pandas as pd
import numpy as np
import scipy as sp

import os
from io import BytesIO
from google.cloud import storage

In [5]:
#Ubicación archivos
client = storage.Client()
bucket = client.get_bucket("mybucket")
ruta1 = "Data"+os.sep+"DANE"+os.sep+"Fuentes_Originales"+os.sep+"Tasa de desempleo"+os.sep
ruta2 = "Data"+os.sep+"DANE"+os.sep+"TASADESEMPLEO"+os.sep 

# Condición de empleo

El siguiente notebook crea una base de datos para observar la condición de empleo en base al archivo anexo_empleo_jun_21.xlsx, documento de Microsoft Excel, sacado de  https://www.dane.gov.co/. https://www.dane.gov.co/index.php/estadisticas-por-tema/mercado-laboral/empleo-y-desempleo/mercado-laboral-historicos.

Este Excel resume: "La Gran encuesta integrada de hogares es una encuesta mediante la cual se solicita información sobre las condiciones de empleo de las personas (si trabajan, en qué trabajan, cuánto ganan, si tienen seguridad social en salud o si están buscando empleo), además de las características generales de la población como sexo, edad, estado civil y nivel educativo, se pregunta sobre sus fuentes de ingresos." $[1]$

El notebook va a generar dos bases de datos: 

1. $\textbf{Desempleo_Ocupacional}$, donde  muestra la población ocupada según posición ocupacional, estas siendo obrero (empleado particular), obrero (empleado del gobierno), empleado doméstico, trabajador por propia cuenta, patrón o empleador, trabajador familiar sin renumeración, trabajador sin remuneración en empresas de otros hogares, jornalero o peón y otros.

2. $\textbf{Desempleo_Actividad}$, donde  muestra la población ocupada según ramas de actividad, estas siendo agricultura, desconocida (no informa), el sector agropecuario (ganadería, caza, silvicultura y pesca), explotación de minas y canteras, industria manufacturera, suministro de electricidad, gas y agua, construcción, comercios (comercio, hoteles y restaurantes), sector (transporte, almacenamiento y comunicaciones), intermediacion financiera, sector (actividades inmobiliarias, empresariales y de alquiler), servicios (comunales, sociales y personales).	

Ambos toman información en Bogotá, Medellín - Valle de Aburrá, Cali - Yumbo, Barranquilla - Soledad, Bucaramanga, Girón, Piedecuesta y Floridablanca, Manizales y Villa María, Pasto, Pereira, Dos Quebradas y La Virginia.

[1] DANE –Departamento Administrativo Nacional de Estadística– (2021). Identificador Mercado Laboral. https://www.dane.gov.co/. https://www.dane.gov.co/index.php/estadisticas-por-tema/mercado-laboral/empleo-y-desempleo/mercado-laboral-historicos

# leer Archivo (segun rama de Actividad)

In [1]:
# Para leer el Excel, se tiene el nombre de las ciudades y en que columna y fila estan

mesescolm = ["A,N:Y","A,Z:AK","A,AL:AW","A,AX:BI","A,BJ:BU","A,BV:CG","A,CH:CS","A,CT:DE","A,DF:DQ","A,DR:EC","A,ED:EO","A,EP:FA","A,FB:FK"]

ciudades = ['Toda Colombia','medellin','barranquilla','bogota','cartagena','manizales','monteria','villavicencio',
           'pasto','cucuta','pereira','bucaramanga','ibague','cali','tunja','florencia','popayan',
           'valledupar','quibdo','neiva','riohacha','santa marta','armenia','sincelejo']

index_ciudad = [13,31,49,67,85,103,121,139,157,175,193,211,229,247,265,283,301,319,337,355,373,
               391,409,427]

In [8]:
Desempleo_Actividad = pd.DataFrame()

archivo = ruta1+'Data_DANE_TASADESEMPLEO_anexo_empleo_jun_21.xlsx'
print(archivo)
crearblob = bucket.get_blob(archivo)
descargarlo = crearblob.download_as_bytes()
nombre_hoja_excel = 'ocu ramas trim 23 áreas' 

#Se crea un for para descargar y concatenar las diferentes bases de datos

for j in range(len(ciudades)):
    year = 2007
    for i in range(len(mesescolm)):
        
        #Los ultimos meses del ultimo año en excel es diferente a los demás, por lo que se pone un if para los dos casos
        
        if (mesescolm[i] == "A,FB:FK"):
                  
            
            Desempleo_Actividadtemp = pd.read_excel(BytesIO(descargarlo), header=index_ciudad[j],sheet_name=nombre_hoja_excel, nrows= 13, usecols = mesescolm[i])
            Desempleo_Actividadtemp.drop(index=Desempleo_Actividadtemp.index[0],axis=0,inplace=True)
            Desempleo_Actividadtemp['Nov - Ene']= np.nan
            Desempleo_Actividadtemp['Dic - Feb']= np.nan
            
            Desempleo_Actividadtemp = Desempleo_Actividadtemp.T
            Desempleo_Actividadtemp = Desempleo_Actividadtemp.reset_index(drop=True)
            
            Desempleo_Actividadtemp.columns = ['Ocupados','No informa','Agricultura, ganadería, caza, silvicultura y pesca',
                                           'Explotación de minas y canteras','Industria manufacturera',
                                           'Suministro de electricidad, gas y agua','Construccion',
                                           'Comercio, hoteles y restaurantes','Transporte, almacenamiento y comunicaciones',
                                           'Intermediacion financiera','Actividades inmobiliarias, empresariales y de alquiler',
                                           'Servicios comunales, sociales y personales']
        
            Desempleo_Actividadtemp.drop(index=Desempleo_Actividadtemp.index[0],axis=0,inplace=True) 
            Desempleo_Actividadtemp['ANO']= year
            Desempleo_Actividadtemp['ciudad']= ciudades[j]
            
            Desempleo_Actividadtemp['mes'] = ['Ene - Mar','Feb - Abr','Mar- May','Abr - Jun','May - Jul','Jun - Ago','Jul - Sep',
                                               'Ago - Oct','Sep - Nov','Oct - Dic','Nov - Ene','Dic - Feb']

            
            Desempleo_Actividad = Desempleo_Actividadtemp.append(Desempleo_Actividad, sort=False)
            year = year+1
        else:
            Desempleo_Actividadtemp = pd.read_excel(BytesIO(descargarlo), header=index_ciudad[j],sheet_name=nombre_hoja_excel, nrows= 13, usecols = mesescolm[i])
            
            Desempleo_Actividadtemp.drop(index=Desempleo_Actividadtemp.index[0],axis=0,inplace=True)
    
            Desempleo_Actividadtemp = Desempleo_Actividadtemp.T
            Desempleo_Actividadtemp = Desempleo_Actividadtemp.reset_index(drop=True)

            Desempleo_Actividadtemp.columns = ['Ocupados','No informa','Agricultura, ganadería, caza, silvicultura y pesca',
                                               'Explotación de minas y canteras','Industria manufacturera',
                                               'Suministro de electricidad, gas y agua','Construccion',
                                               'Comercio, hoteles y restaurantes','Transporte, almacenamiento y comunicaciones',
                                               'Intermediacion financiera','Actividades inmobiliarias, empresariales y de alquiler',
                                               'Servicios comunales, sociales y personales']

            Desempleo_Actividadtemp.drop(index=Desempleo_Actividadtemp.index[0],axis=0,inplace=True)  

            Desempleo_Actividadtemp['ANO']= year
            Desempleo_Actividadtemp['ciudad']= ciudades[j]
            Desempleo_Actividadtemp['mes'] = ['Ene - Mar','Feb - Abr','Mar- May','Abr - Jun','May - Jul','Jun - Ago','Jul - Sep',
                                                   'Ago - Oct','Sep - Nov','Oct - Dic','Nov - Ene','Dic - Feb']


            Desempleo_Actividad = Desempleo_Actividadtemp.append(Desempleo_Actividad, sort=False)
            year = year+1


Data/DANE/Fuentes_Originales/Tasa de desempleo/Data_DANE_TASADESEMPLEO_anexo_empleo_jun_21.xlsx


In [9]:
Desempleo_Actividad.head()

,Ocupados,No informa,"Agricultura, ganadería, caza, silvicultura y pesca",Explotación de minas y canteras,Industria manufacturera,"Suministro de electricidad, gas y agua",Construccion,"Comercio, hoteles y restaurantes","Transporte, almacenamiento y comunicaciones",Intermediacion financiera,"Actividades inmobiliarias, empresariales y de alquiler","Servicios comunales, sociales y personales",ANO,ciudad,mes
1,130.442,0.029667,1.604,0.071,14.915333,0.839333,9.608333,47.826333,15.840333,1.682333,6.475667,31.55,2019,sincelejo,Ene - Mar
2,130.894667,0.029667,1.802667,0.165333,14.722,0.952667,9.489,46.254333,16.755667,1.892667,6.382333,32.448333,2019,sincelejo,Feb - Abr
3,131.217667,0.029667,1.753,0.189333,14.078,1.188667,9.448667,45.081667,17.301,1.953,6.658,33.537333,2019,sincelejo,Mar- May
4,131.092,0.0,1.645333,0.152,13.398667,1.498,9.319333,45.459,16.862,2.216333,5.484,35.057,2019,sincelejo,Abr - Jun
5,130.864,0.0,1.650667,0.334667,13.837667,1.237333,9.448,45.517667,16.245667,2.24,5.315667,35.036,2019,sincelejo,May - Jul


# Descarga con actividad

In [8]:
Desempleo_Actividad.to_parquet('tmp.parquet',index = False) 
archivo = ruta2+ 'Desempleo_Actividad.parquet' 

file_name_local='tmp.parquet'
bucket.blob(archivo).upload_from_filename(file_name_local)

# leer Archivo (según posición ocupacional)

In [10]:
mesescolm = ["A:M","A,N:Y","A,Z:AK","A,AL:AW","A,AX:BI","A,BJ:BU","A,BV:CG","A,CH:CS","A,CT:DE","A,DF:DQ","A,DR:EC","A,ED:EO","A,EP:FA",
             "A,FB:FM","A,FN:FY","A,FZ:GC"]

ciudades = ['total','medellin','barranquilla','bogota','cartagena','manizales','monteria','villavicencio',
           'pasto','cucuta','pereira','bucaramanga','ibague','cali','tunja','florencia','popayan',
           'valledupar','quibdo','neiva','riohacha','santa marta','armenia','sincelejo']

index_ciudad = [13,29,45,61,77,93,109,125,141,157,173,189,205,221,237,253,269,285,301,317,333,349,
               365,381]

In [11]:
Desempleo_Ocupacional = pd.DataFrame()


archivo = ruta1+'Data_DANE_TASADESEMPLEO_anexo_empleo_jun_21.xlsx'
print(archivo)
crearblob = bucket.get_blob(archivo)
descargarlo = crearblob.download_as_bytes()

nombre_hoja_excel = 'ocu posc trim 23 áreas' 

for j in range(len(ciudades)):
    year = 2007
    #print(ciudades[j])
    for i in range(len(mesescolm)):
        if (mesescolm[i] == "A,FZ:GC"):
                  
            
            Desempleo_Ocupacionaltemp = pd.read_excel(BytesIO(descargarlo), header=index_ciudad[j],sheet_name=nombre_hoja_excel, nrows= 11, usecols = mesescolm[i])
            
            Desempleo_Ocupacionaltemp.drop(index=Desempleo_Ocupacionaltemp.index[0],axis=0,inplace=True)
            
            Desempleo_Ocupacionaltemp['May - Jul']= np.nan
            Desempleo_Ocupacionaltemp['Jun - Ago']= np.nan
            Desempleo_Ocupacionaltemp['Jul - Sep']= np.nan
            Desempleo_Ocupacionaltemp['Ago - Oct']= np.nan
            Desempleo_Ocupacionaltemp['Sep - Nov']= np.nan
            Desempleo_Ocupacionaltemp['Oct - Dic']= np.nan
            Desempleo_Ocupacionaltemp['Nov - Ene']= np.nan
            Desempleo_Ocupacionaltemp['Dic - Feb']= np.nan
            
            Desempleo_Ocupacionaltemp = Desempleo_Ocupacionaltemp.T
            Desempleo_Ocupacionaltemp = Desempleo_Ocupacionaltemp.reset_index(drop=True)
            
            Desempleo_Ocupacionaltemp.columns = ['Ocupados','Obrero, empleado particular ',
                                           'Obrero, empleado del gobierno','Empleado doméstico',
                                           'Trabajador por cuenta propia','Patron o empleador',
                                           'Trabajador familiar sin remuneracion',
                                           'Trabajador sin remuneración en empresas de otros hogares',
                                           'Jornalero o Peón','Otro']
        
            Desempleo_Ocupacionaltemp.drop(index=Desempleo_Ocupacionaltemp.index[0],axis=0,inplace=True) 
            Desempleo_Ocupacionaltemp['ANO']= year
            Desempleo_Ocupacionaltemp['ciudad']= ciudades[j]
            
            Desempleo_Ocupacionaltemp['mes'] = ['Ene - Mar','Feb - Abr','Mar- May','Abr - Jun','May - Jul','Jun - Ago','Jul - Sep',
                                               'Ago - Oct','Sep - Nov','Oct - Dic','Nov - Ene','Dic - Feb']

            
            Desempleo_Ocupacional = Desempleo_Ocupacionaltemp.append(Desempleo_Ocupacional, sort=False)
            year = year+1
        else:
            Desempleo_Ocupacionaltemp = pd.read_excel(BytesIO(descargarlo), header=index_ciudad[j],sheet_name=nombre_hoja_excel, nrows= 11, usecols = mesescolm[i])
            
            Desempleo_Ocupacionaltemp.drop(index=Desempleo_Ocupacionaltemp.index[0],axis=0,inplace=True)
    
            Desempleo_Ocupacionaltemp = Desempleo_Ocupacionaltemp.T
            Desempleo_Ocupacionaltemp = Desempleo_Ocupacionaltemp.reset_index(drop=True)

            Desempleo_Ocupacionaltemp.columns = ['Ocupados','Obrero, empleado particular ',
                                           'Obrero, empleado del gobierno','Empleado doméstico',
                                           'Trabajador por cuenta propia','Patron o empleador',
                                           'Trabajador familiar sin remuneracion',
                                           'Trabajador sin remuneración en empresas de otros hogares',
                                           'Jornalero o Peón','Otro']
            
            Desempleo_Ocupacionaltemp.drop(index=Desempleo_Ocupacionaltemp.index[0],axis=0,inplace=True)  

            Desempleo_Ocupacionaltemp['ANO']= year
            Desempleo_Ocupacionaltemp['ciudad']= ciudades[j]
            Desempleo_Ocupacionaltemp['mes'] = ['Ene - Mar','Feb - Abr','Mar- May','Abr - Jun','May - Jul','Jun - Ago','Jul - Sep',
                                                   'Ago - Oct','Sep - Nov','Oct - Dic','Nov - Ene','Dic - Feb']


            Desempleo_Ocupacional = Desempleo_Ocupacionaltemp.append(Desempleo_Ocupacional, sort=False)
            year = year+1

Data/DANE/Fuentes_Originales/Tasa de desempleo/Data_DANE_TASADESEMPLEO_anexo_empleo_jun_21.xlsx


In [13]:
Desempleo_Ocupacional.head(3)

,Ocupados,"Obrero, empleado particular","Obrero, empleado del gobierno",Empleado doméstico,Trabajador por cuenta propia,Patron o empleador,Trabajador familiar sin remuneracion,Trabajador sin remuneración en empresas de otros hogares,Jornalero o Peón,Otro,ANO,ciudad,mes
1,116.608333,28.276667,6.651667,3.918667,68.722333,2.548667,5.974333,0.439,0.077333,0.0,2022,sincelejo,Ene - Mar
2,116.344333,27.099333,6.932,4.348,68.716667,2.859,5.803,0.406333,0.141,0.039667,2022,sincelejo,Feb - Abr
3,114.834333,27.918667,6.791,4.298667,66.932667,2.427333,5.538667,0.667667,0.221,0.039667,2022,sincelejo,Mar- May


In [14]:
Desempleo_Ocupacional.columns

Index(['Ocupados', 'Obrero, empleado particular ',
       'Obrero, empleado del gobierno', 'Empleado doméstico',
       'Trabajador por cuenta propia', 'Patron o empleador',
       'Trabajador familiar sin remuneracion',
       'Trabajador sin remuneración en empresas de otros hogares',
       'Jornalero o Peón', 'Otro', 'ANO', 'ciudad', 'mes'],
      dtype='object')

In [15]:
Desempleo_Ocupacional['Ocupados'] = Desempleo_Ocupacional['Ocupados'].astype(str)
Desempleo_Ocupacional['Obrero, empleado particular '] = Desempleo_Ocupacional['Obrero, empleado particular '].astype(str)
Desempleo_Ocupacional['Obrero, empleado del gobierno'] = Desempleo_Ocupacional['Obrero, empleado del gobierno'].astype(str)
Desempleo_Ocupacional['Empleado doméstico'] = Desempleo_Ocupacional['Empleado doméstico'].astype(str)
Desempleo_Ocupacional['Trabajador por cuenta propia'] = Desempleo_Ocupacional['Trabajador por cuenta propia'].astype(str)
Desempleo_Ocupacional['Patron o empleador'] = Desempleo_Ocupacional['Patron o empleador'].astype(str)
Desempleo_Ocupacional['Trabajador familiar sin remuneracion'] = Desempleo_Ocupacional['Trabajador familiar sin remuneracion'].astype(str)
Desempleo_Ocupacional['Jornalero o Peón'] = Desempleo_Ocupacional['Jornalero o Peón'].astype(str)
Desempleo_Ocupacional['Otro'] = Desempleo_Ocupacional['Otro'].astype(str)
Desempleo_Ocupacional['ANO'] = Desempleo_Ocupacional['ANO'].astype(str)
Desempleo_Ocupacional['ciudad'] = Desempleo_Ocupacional['ciudad'].astype(str)
Desempleo_Ocupacional['mes'] = Desempleo_Ocupacional['mes'].astype(str)

In [16]:
Desempleo_Ocupacional['Trabajador sin remuneración en empresas de otros hogares'] = Desempleo_Ocupacional['Trabajador sin remuneración en empresas de otros hogares'].map(str)
Desempleo_Ocupacional.head(3)

,Ocupados,"Obrero, empleado particular","Obrero, empleado del gobierno",Empleado doméstico,Trabajador por cuenta propia,Patron o empleador,Trabajador familiar sin remuneracion,Trabajador sin remuneración en empresas de otros hogares,Jornalero o Peón,Otro,ANO,ciudad,mes
1,116.60833333333335,28.276666666666667,6.651666666666667,3.9186666666666667,68.72233333333332,2.5486666666666666,5.974333333333334,0.439,0.07733333333333334,0.0,2022,sincelejo,Ene - Mar
2,116.34433333333334,27.099333333333334,6.9319999999999995,4.348,68.71666666666665,2.859,5.803,0.40633333333333327,0.14100000000000001,0.03966666666666666,2022,sincelejo,Feb - Abr
3,114.83433333333335,27.918666666666667,6.7909999999999995,4.298666666666667,66.93266666666666,2.4273333333333333,5.538666666666667,0.6676666666666667,0.221,0.03966666666666666,2022,sincelejo,Mar- May


# Descarga con ocupación

In [33]:
Desempleo_Ocupacional.to_parquet('tmp.parquet',index = False) 
archivo = ruta2+ 'Desempleo_Ocupacional.parquet' 

file_name_local='tmp.parquet'
bucket.blob(archivo).upload_from_filename(file_name_local)